In [15]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import GridSearchCV, cross_val_score

# load corpus
corpus = pd.read_csv('corpus.csv.gz', compression='gzip')
corpus_feat = pd.read_csv('corpus_liwc_mtx.csv.gz', compression='gzip')
corpus_feat.shape

(1000, 67)

In [16]:
corpus_feat['class'] = corpus['qual_a_melhor_classificao_para_esse_texto']
corpus_feat['confidence'] = corpus['qual_a_melhor_classificao_para_esse_texto:confidence']
corpus_feat['wc'] = corpus['contentcount']

In [17]:
corpus_feat.drop(corpus_feat.columns[0], axis=1,inplace=True)
corpus_feat.drop(corpus_feat.columns[0], axis=1,inplace=True)
corpus_feat = corpus_feat[corpus_feat.wc.apply(lambda x: str(x).isnumeric())]

wc_vector = corpus_feat['wc']
class_vector = corpus_feat['class']

corpus_feat.drop('class',axis=1,inplace=True)
corpus_feat.drop('confidence',axis=1,inplace=True)
corpus_feat.drop('wc',axis=1,inplace=True)

In [28]:
data = corpus_feat.as_matrix().astype(float) / wc_vector.as_matrix().astype(float)[:, np.newaxis]
data[np.isnan(data)] = 0
data[data >= 1E308] = 0
data.shape

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in true_divide
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


(997, 64)

In [29]:
# fix labels to binary
from sklearn import preprocessing
lb = preprocessing.LabelBinarizer(neg_label=1, pos_label=2)
target = lb.fit_transform(class_vector.values)
c, r = target.shape
target = target.reshape(c,)

## Evaluating SVM

In [30]:
parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
grid_search = GridSearchCV(SVC(), parameters, cv=3, n_jobs=3, verbose=3, scoring='accuracy')

grid_search.fit(data, target)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] C=1, kernel=linear ..............................................
[CV] C=1, kernel=linear ..............................................
[CV] C=1, kernel=linear ..............................................
[CV] ............... C=1, kernel=linear, score=0.636637, total=   0.1s
[CV] C=1, kernel=rbf .................................................
[CV] ............... C=1, kernel=linear, score=0.635542, total=   0.1s
[CV] C=1, kernel=rbf .................................................
[CV] ............... C=1, kernel=linear, score=0.635542, total=   0.1s
[CV] C=1, kernel=rbf .................................................
[CV] .................. C=1, kernel=rbf, score=0.635542, total=   0.1s
[CV] C=10, kernel=linear .............................................
[CV] .................. C=1, kernel=rbf, score=0.635542, total=   0.1s
[CV] C=10, kernel=linear .............................................
[CV] ............

[Parallel(n_jobs=3)]: Done  12 out of  12 | elapsed:    0.7s remaining:    0.0s
[Parallel(n_jobs=3)]: Done  12 out of  12 | elapsed:    0.7s finished


GridSearchCV(cv=3, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=3,
       param_grid={'kernel': ('linear', 'rbf'), 'C': [1, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='accuracy', verbose=3)

In [31]:
print(grid_search.best_estimator_)
print(grid_search.best_score_)
print(grid_search.best_params_)

SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
0.637913741224
{'C': 10, 'kernel': 'linear'}


## Evaluating Naive Bayses

In [32]:
from sklearn.naive_bayes import GaussianNB

model = GaussianNB()

acc = cross_val_score(model, data, target, cv=10, scoring='accuracy').mean()
recall = cross_val_score(model, data, target, cv=10, scoring='recall').mean()
print(acc)
print(recall)

0.369113311331
0.0142113095238


In [33]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()

f1 = cross_val_score(model, data, target, cv=10, scoring='f1').mean()
acc = cross_val_score(model, data, target, cv=10, scoring='accuracy').mean()
recall = cross_val_score(model, data, target, cv=10, scoring='recall').mean()
print(f1)
print(acc)
print(recall)

0.776683575644
0.634907090709
0.998412698413
